In [1]:
import requests
import env
from requests.models import Request, Response
from env import Client_ID
import pandas as pd
from igdb.igdbapi_pb2 import GameResult
from typing import Dict, List, Optional, Union, cast
import acquire
import json
from igdb.wrapper import IGDBWrapper

### Connect to API and scrape the data needed
#### get the access token and 200 response 

In [ ]:
def get_api():   
    url = env.get_db_url()
    response = requests.post(url)
    data = response.json()
    access_token = data['access_token']
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *;'
    r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)
    return access_token, r

In [2]:
access_token = acquire.connect_api()

In [3]:
access_token

'w1fejmtqp5gyuy747q28ttwjzgybbh'

In [4]:
tf = acquire.connect_to_games()

In [5]:
tf

<Response [200]>

In [6]:
wrapper = acquire.run_wrapper()

In [7]:
wrapper

In [8]:
game_library = acquire.get_game_library(wrapper)
game_library.head()

,id,age_ratings,artworks,category,cover,created_at,external_games,first_release_date,genres,name,...,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,192310,"[63703, 63704]",[68886],0,210891.0,1646209573,"[2270833, 2272174, 2326733, 2591087]",1.645661e+09,[31],Martha Is Dead: Digital Deluxe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,NaN,0,NaN,1498435200,"[20127, 1988884]",5.364576e+08,[5],Dogou Souken,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,[37944],NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[9],City Mysteries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[31, 32]",Sword of the Black Stone,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,NaN,[33736],0,169077.0,1621959514,"[2052375, 2057300]",NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.platforms.tail(10).values[0]

## scraping endpoints
scrape the end points of each category and add them to the game_librarys

### platform_families

In [ ]:
def get_platform_families():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_families' ,data=data ,  headers=headers)
    platform_families = wrapper.api_request('platform_families', 'fields *;limit 500;')
    x = json.loads(platform_families)
    platform_families =pd.DataFrame(x)
    return platform_families

In [ ]:
platform_families = get_platform_families()
platform_families.head()

In [ ]:
df.columns

### platform_version_release_dates

In [ ]:
def get_platform_version_release():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_version_release_dates' ,data=data ,  headers=headers)
    platform_version_release_dates = wrapper.api_request('platform_version_release_dates', 'fields *;limit 500;')
    x = json.loads(platform_version_release_dates)
    platform_version_release_dates =pd.DataFrame(x)
    return platform_version_release_dates

In [ ]:
platform_version_release_dates = get_platform_version_release()
platform_version_release_dates

In [ ]:
# drop columns not needed
platform_version_release_dates = platform_version_release_dates.drop(columns=['category', 'date', 'm', 'y', 'region', 'updated_at', 'checksum', 'updated_at'])
platform_version_release_dates.head()


In [ ]:
df.head()

In [ ]:
version_dates_dict[9] = 'not_available'
version_dates_dict[384] = 'not_available'
version_dates_dict[386] = 'not_available'


In [ ]:
version_dates_list = platform_version_release_dates[['game' , 'human']].sort_values(by='game').reset_index(drop=True)
version_dates_dict = platform_version_release_dates.set_index('game').to_dict()['human']

In [ ]:
def test_funct3(random_list):
    if type(random_list) == list:
        return [version_dates_dict[i] for i in random_list]
    else:  
        return "Not available"

In [ ]:
platforms.set_index('Locality', inplace=True)
data.set_index('Locality', inplace=True)


In [ ]:
df['release_dates'] = df.release_dates.apply(test_funct3)
df.head()

In [ ]:
all_release_dates = pd.merge(release_dates, platform_version_release_dates, how = 'inner', on="human")

In [ ]:
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df.head()

### platform_version_companies

In [ ]:
def get_platform_version_companies():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_version_companies' ,data=data ,  headers=headers)
    platform_version_companies = wrapper.api_request('platform_version_companies', 'fields *;limit 500;')
    x = json.loads(platform_version_companies)
    platform_version_companies =pd.DataFrame(x)
    return platform_version_companies

In [ ]:
platform_version_companies=get_platform_version_companies()
platform_version_companies.head()

### platforms (use this)

In [9]:
def get_platforms():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platforms' ,data=data ,  headers=headers)
    platforms = wrapper.api_request('platforms', 'fields *;limit 500;')
    x = json.loads(platforms)
    platforms =pd.DataFrame(x)
    return platforms

In [10]:
platform = get_platforms()
platform.shape

(197, 16)

In [ ]:
# see if there is a number in id that matches the big df
platform.loc[platform['id'] == 5]

In [ ]:
platform = platform.drop(columns=['alternative_name', 'category', 'created_at', 'platform_logo', 'updated_at', 'url', 'checksum', 'versions', 'websites', 'generation', 'platform_family', 'abbreviation', 'summary'])
platform.head()


In [ ]:
# looking at the dictionary
print(game_library.get(92, "Not found"))

In [ ]:
df.platforms.iloc[2]

In [11]:
platformlist = platform[['id' , 'name']].sort_values(by='id').reset_index(drop=True)


In [12]:
platform_dict = platformlist.set_index('id').to_dict()['name']

In [13]:
platform_dict[92] = 'not_available'

In [14]:
def test_funct(random_list):
    if type(random_list) == list:
        return [platform_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [15]:
game_library['platforms'] = game_library.platforms.apply(test_funct)
game_library.head()

,id,age_ratings,artworks,category,cover,created_at,external_games,first_release_date,genres,name,...,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,192310,"[63703, 63704]",[68886],0,210891.0,1646209573,"[2270833, 2272174, 2326733, 2591087]",1.645661e+09,[31],Martha Is Dead: Digital Deluxe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,NaN,0,NaN,1498435200,"[20127, 1988884]",5.364576e+08,[5],Dogou Souken,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,[37944],NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[9],City Mysteries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[31, 32]",Sword of the Black Stone,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,NaN,[33736],0,169077.0,1621959514,"[2052375, 2057300]",NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.loc[df['platforms'] == '3ds']

In [ ]:
df.loc[df['platforms'].notnull()]

### release_dates

In [ ]:
def get_release_dates():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/release_dates' ,data=data ,  headers=headers)
    release_dates = wrapper.api_request('release_dates', 'fields *;limit 500;')
    x = json.loads(release_dates)
    release_dates =pd.DataFrame(x)
    return release_dates

In [ ]:
release_dates = get_release_dates()
release_dates.shape

In [ ]:
release_dates = release_dates.drop(columns=['category', 'date', 'm', 'region', 'updated_at', 'y', 'checksum', 'created_at', 'platform'])
release_dates.head()

In [ ]:
release_dates

In [ ]:
# add to dictionary
release_dict[91007] = "not-available"
release_dict[138333] = "not-available"
release_dict[188385] = "not-available"
release_dict[213594] = "not-available"
release_dict[77755] = "not-available"


In [ ]:
releaselist = release_dates[['game' , 'human']].sort_values(by='game').reset_index(drop=True)
release_dict = releaselist.set_index('game').to_dict()['human']

In [ ]:
release_dates.head()

In [ ]:
def test_funct2(random_list):
    if type(random_list) == list:
        return [release_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [ ]:
release_dict

In [ ]:
df.release_dates.head()

In [ ]:
df['release_dates'] = pd.DataFrame(df.release_dates.apply(test_funct2))
df

In [ ]:
release_dict

In [ ]:
# add 0 and description to dictionary
release_dict["0"] = "not-available"

### platform_versions

In [ ]:
def get_platform_versions():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/platform_versions' ,data=data ,  headers=headers)
    platform_versions = wrapper.api_request('platform_versions', 'fields *;limit 500;')
    x = json.loads(platform_versions)
    platform_versions=pd.DataFrame(x)
    return platform_versions

In [ ]:
platform_versions = get_platform_versions()
platform_versions.shape

In [ ]:
platform_versions.head()

In [ ]:
df.columns

### player_perspectives (use this)

In [16]:
def get_player_perspectives():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/player_perspectives' ,data=data ,  headers=headers)
    player_perspectives = wrapper.api_request('player_perspectives', 'fields *;limit 500;')
    x = json.loads(player_perspectives)
    player_perspectives=pd.DataFrame(x)
    return player_perspectives

In [17]:
player_perspectives = get_player_perspectives()
player_perspectives.shape

(7, 7)

In [ ]:
player_perspectives = player_perspectives.drop(columns=['created_at','updated_at', 'url', 'checksum'])
player_perspectives.head()


In [18]:
player_perspectives_list = player_perspectives[['id' , 'name']].sort_values(by='id').reset_index(drop=True)
player_perspectives_dict = player_perspectives.set_index('id').to_dict()['name']

In [19]:
def test_funct4(random_list):
    if type(random_list) == list:
        return [player_perspectives_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [20]:
game_library['player_perspectives'] = pd.DataFrame(game_library.player_perspectives.apply(test_funct4))
game_library.head()

,id,age_ratings,artworks,category,cover,created_at,external_games,first_release_date,genres,name,...,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,192310,"[63703, 63704]",[68886],0,210891.0,1646209573,"[2270833, 2272174, 2326733, 2591087]",1.645661e+09,[31],Martha Is Dead: Digital Deluxe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,NaN,0,NaN,1498435200,"[20127, 1988884]",5.364576e+08,[5],Dogou Souken,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,[37944],NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[9],City Mysteries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[31, 32]",Sword of the Black Stone,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,NaN,[33736],0,169077.0,1621959514,"[2052375, 2057300]",NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### themes (use this)

In [21]:
def get_themes():
    headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
    data = 'fields *; '
    r = requests.post('https://api.igdb.com/v4/themes' ,data=data ,  headers=headers)
    themes = wrapper.api_request('themes', 'fields *;limit 500;')
    x = json.loads(themes)
    themes=pd.DataFrame(x)
    return themes

In [22]:
themes = get_themes()
themes.shape

(22, 7)

In [23]:
themes_list = themes[['id' , 'name']].sort_values(by='id').reset_index(drop=True)
themes_dict = themes.set_index('id').to_dict()['name']

In [24]:
def test_funct5(random_list):
    if type(random_list) == list:
        return [themes_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [25]:
game_library['themes'] = pd.DataFrame(game_library.themes.apply(test_funct5))
game_library.head()

,id,age_ratings,artworks,category,cover,created_at,external_games,first_release_date,genres,name,...,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,192310,"[63703, 63704]",[68886],0,210891.0,1646209573,"[2270833, 2272174, 2326733, 2591087]",1.645661e+09,[31],Martha Is Dead: Digital Deluxe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,NaN,0,NaN,1498435200,"[20127, 1988884]",5.364576e+08,[5],Dogou Souken,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,[37944],NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[9],City Mysteries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[31, 32]",Sword of the Black Stone,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,NaN,[33736],0,169077.0,1621959514,"[2052375, 2057300]",NaN,"[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Genres (use this)

In [26]:
# function that puts response list object into a dataframe for each page
def get_genres(wrapper):
    genres = pd.DataFrame()
    for i in range (0, 409):
        genre = wrapper.api_request('genres', 'fields *; limit 500;')
        y = json.loads(genre)
        results_df =pd.DataFrame(y)
        genres = pd.concat([genres, results_df])
    return genres

In [27]:
genres = get_genres(wrapper)
genres.head()

,id,created_at,name,slug,updated_at,url,checksum
0,4,1297555200,Fighting,fighting,1323216000,https://www.igdb.com/genres/fighting,2ccc6572-bdde-6ed4-8843-25447ea40782
1,5,1297555200,Shooter,shooter,1323216000,https://www.igdb.com/genres/shooter,bb15fd3f-0f46-e5f3-2b40-d046cf9bd2ef
2,7,1297555200,Music,music,1323216000,https://www.igdb.com/genres/music,22d44a0d-89c7-580f-eef2-e06f178fdd47
3,8,1297555200,Platform,platform,1323216000,https://www.igdb.com/genres/platform,697fc5a4-b96f-a803-288a-498bd5dd1de1
4,9,1297555200,Puzzle,puzzle,1323216000,https://www.igdb.com/genres/puzzle,616de9c3-8a00-0232-9df9-00014cfac51b


In [28]:
genreslist = genres[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)
genres_dict = genreslist.set_index('id').to_dict()['slug']

In [29]:
def test_functg(random_list):
    if type(random_list) == list:
        return [genres_dict[i] for i in random_list]
    else:  
        return  ["Not available"] 

In [30]:
game_library['genres'] = pd.DataFrame(game_library.genres.apply(test_functg))
game_library.head()

,id,age_ratings,artworks,category,cover,created_at,external_games,first_release_date,genres,name,...,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,192310,"[63703, 63704]",[68886],0,210891.0,1646209573,"[2270833, 2272174, 2326733, 2591087]",1.645661e+09,[adventure],Martha Is Dead: Digital Deluxe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,NaN,0,NaN,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,[37944],NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,NaN,[33736],0,169077.0,1621959514,"[2052375, 2057300]",NaN,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Game modes (use this)

In [31]:
def get_game_modes():
     headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
     data = 'fields *; limit 500;'
     r = requests.post('https://api.igdb.com/v4/game_modes' ,data=data ,  headers=headers)
     game_modes = wrapper.api_request('game_modes', 'fields *;limit 500;')
     x = json.loads(game_modes)
     game_modes =pd.DataFrame(x)
     return game_modes

In [32]:
game_mode = get_game_modes()

In [33]:
modelist = game_mode[['id' , 'slug']].sort_values(by='id').reset_index(drop=True)
mode_dict = modelist.set_index('id').to_dict()['slug']

In [34]:
def test_functm(random_list):
    if type(random_list) == list:
        return [mode_dict[i] for i in random_list]
    else:  
        return  ["Not available"]

In [35]:
game_library['game_modes'] = pd.DataFrame(game_library.game_modes.apply(test_functm))
game_library.head()

,id,age_ratings,artworks,category,cover,created_at,external_games,first_release_date,genres,name,...,aggregated_rating_count,multiplayer_modes,remakes,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions
0,192310,"[63703, 63704]",[68886],0,210891.0,1646209573,"[2270833, 2272174, 2326733, 2591087]",1.645661e+09,[adventure],Martha Is Dead: Digital Deluxe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,NaN,0,NaN,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,[37944],NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,NaN,[33736],0,169077.0,1621959514,"[2052375, 2057300]",NaN,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Game version features (use this)

In [36]:
def get_game_version_features(wrapper):
    game_version_features = pd.DataFrame()
    for i in range (0, 409):
        a_ratings = wrapper.api_request('game_version_features', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(a_ratings)
        results_df =pd.DataFrame(y)
        game_version_features = pd.concat([game_version_features, results_df])
    return game_version_features

In [37]:
game_version_features = get_game_version_features(wrapper)
game_version_features.head()

,id,category,description,position,title,values,checksum
0,337,0.0,Sharpens your aim with rifles and pistols. Alw...,1,DLC: The Sharpshooter Detective Suit,"[1385, 1386, 1387, 1388]",a10d3ebf-ae3a-f0cc-fc68-116b1545a9ac
1,347,0.0,NaN,0,"25 cm (10"") figurine of an Alb Mage","[1419, 1420]",2a42989b-4936-75bc-6331-0ce79abb225a
2,352,0.0,NaN,5,A making of DVD,"[1429, 1430]",bb46d55f-7145-0b5c-2f69-cea9dd2570d6
3,375,0.0,Double sided print of the in-game island,2,Collectible map,"[1519, 1520, 1521, 1522, 1523]",8a9ea6d0-b5f1-ece2-cd5d-61636d639ad0
4,382,0.0,"Exclusive to TESCO in the UK, it included thre...",9,Combat Strike Pack,"[1554, 1555, 1556, 1557, 1558]",263031ce-acb2-69a2-ab66-240dbaad4f4d


In [38]:
game_vf_df = game_version_features[['id','description', 'title']]
game_vf_df.head()

,id,description,title
0,337,Sharpens your aim with rifles and pistols. Alw...,DLC: The Sharpshooter Detective Suit
1,347,NaN,"25 cm (10"") figurine of an Alb Mage"
2,352,NaN,A making of DVD
3,375,Double sided print of the in-game island,Collectible map
4,382,"Exclusive to TESCO in the UK, it included thre...",Combat Strike Pack


In [39]:
game_vf_df = game_vf_df.rename(columns={'description': 'version_description', 'title': 'title_version'})

## Game version feature values (use this)

In [40]:
def vf_values(wrapper):
    vf_values= pd.DataFrame()
    for i in range (0, 409):
        gvf_values = wrapper.api_request('game_version_feature_values', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(gvf_values)
        results_df =pd.DataFrame(y)
        vf_values = pd.concat([vf_values, results_df])
    return vf_values

In [41]:
gamevfv_df = vf_values(wrapper)
gamevfv_df.head()

,id,game,game_feature,included_feature,checksum,note
0,1581,52617,391,0.0,1db62035-9135-3d01-25a5-9c194d8fbd8f,NaN
1,1590,52617,394,0.0,e2e097d5-c88d-d6ea-48cf-1f5288f6bbe2,NaN
2,591,39045,144,0.0,6ee07864-0f1b-d41f-e2a5-e7dc2731d9a0,NaN
3,1987,83886,503,2.0,612d4e1b-a3b2-60e4-680a-064c0837f71e,NaN
4,1600,19333,399,0.0,ba6dcc29-45c7-6412-ebce-da6731910fb9,NaN


In [42]:
gamevfv_df = gamevfv_df[['game','game_feature']]
gamevfv_df.head()

,game,game_feature
0,52617,391
1,52617,394
2,39045,144
3,83886,503
4,19333,399


In [43]:
gamevfv_df = gamevfv_df.rename(columns={"game_feature": "id"})

In [44]:
gamevfv_df = gamevfv_df.set_index('id')
gamevfv_df.head()

,game
id,
391,52617
394,52617
144,39045
503,83886
399,19333


### Merging game feature version and game feature version values (use this)

In [45]:
results_df = pd.merge(gamevfv_df,  game_vf_df , how = 'left', on="id")
results_df.head()

,id,game,version_description,title_version
0,391,52617,"Delve into the mystery of the Shining Lady, Ca...",Field Report with Removable Coin
1,394,52617,Show the world you've joined the siege with th...,Siege of Dragonspear Amulet
2,144,39045,NaN,Bayek’s Eagle Skull Amulet
3,503,83886,Doomsday Prepper Pack includes the prepper out...,Doomsday Prepper Pack
4,399,19333,Prospect Mask with a display base,Prospect Mask


In [46]:
results_df = results_df.rename(columns={'id': 'second','game': 'id' })
results_df = results_df.set_index('id')
results_df.head()

,second,version_description,title_version
id,,,
52617,391,"Delve into the mystery of the Shining Lady, Ca...",Field Report with Removable Coin
52617,394,Show the world you've joined the siege with th...,Siege of Dragonspear Amulet
39045,144,NaN,Bayek’s Eagle Skull Amulet
83886,503,Doomsday Prepper Pack includes the prepper out...,Doomsday Prepper Pack
19333,399,Prospect Mask with a display base,Prospect Mask


In [47]:
#merging the to the main dataframe
game_library = pd.merge(game_library, results_df, how = 'left', on="id")
game_library.head()

,id,age_ratings,artworks,category,cover,created_at,external_games,first_release_date,genres,name,...,dlcs,ports,expansions,expanded_games,remasters,forks,standalone_expansions,second,version_description,title_version
0,192310,"[63703, 63704]",[68886],0,210891.0,1646209573,"[2270833, 2272174, 2326733, 2591087]",1.645661e+09,[adventure],Martha Is Dead: Digital Deluxe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,NaN,0,NaN,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,[37944],NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,NaN,[33736],0,169077.0,1621959514,"[2052375, 2057300]",NaN,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Multi player mode (use this)

In [48]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; limit 500;'
r = requests.post('https://api.igdb.com/v4/multiplayer_modes' ,data=data ,  headers=headers)
multiplayer_modes = wrapper.api_request('multiplayer_modes', 'fields *;limit 500;')
x = json.loads(multiplayer_modes)
multiplayer_modes =pd.DataFrame(x)

In [49]:
multiplayer_modes.head()

,id,campaigncoop,dropin,game,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,checksum,offlinecoopmax,onlinecoopmax,onlinemax
0,9953,False,False,92273,False,False,2.0,False,22.0,False,1b94d937-dc0c-c62d-10af-e58d42860bd8,NaN,NaN,NaN
1,1832,True,True,7153,False,True,0.0,False,12.0,False,2b09f64f-b8b9-6948-ad98-9ab9bede8e3b,2.0,0.0,0.0
2,7987,False,False,57887,False,True,2.0,False,49.0,False,56c9ee4c-aae3-ba3c-ead0-9e899e1c5267,0.0,0.0,0.0
3,7,False,False,46076,False,False,30.0,False,6.0,False,a41bcf9c-976d-8690-629f-4c2737f65bf1,NaN,NaN,NaN
4,10207,False,False,31256,False,False,NaN,False,82.0,False,788009fd-9e08-5826-577b-9844575ba2d9,NaN,NaN,NaN


In [50]:
def multi_play(wrapper):
    multi_play= pd.DataFrame()
    for i in range (0, 409):
        multi_values = wrapper.api_request('multiplayer_modes', 'fields *; limit 500;' f'offset {i * 500};')
        y = json.loads(multi_values)
        results_df =pd.DataFrame(y)
        multi_play = pd.concat([multi_play, multiplayer_modes])
    return multi_play

In [51]:
multi_df = multi_play(wrapper)

In [52]:
def test_functg(random_list):
    if type(random_list) == list:
        return [genres_dict[i] for i in random_list]
    else:  
        return  ["Not available"] 

In [53]:
multi_df = multi_df.rename(columns={'game': 'id', 'id': 'second_id'})

In [54]:
multi_df = multi_df.set_index('id')

In [55]:
#marging this together with the main df
game_library = pd.merge(game_library, multi_df, how = 'left', on="id")
game_library.head()

,id,age_ratings,artworks,category,cover,created_at,external_games,first_release_date,genres,name,...,lancoop,offlinecoop,offlinemax,onlinecoop,platform,splitscreen,checksum_y,offlinecoopmax,onlinecoopmax,onlinemax
0,192310,"[63703, 63704]",[68886],0,210891.0,1646209573,"[2270833, 2272174, 2326733, 2591087]",1.645661e+09,[adventure],Martha Is Dead: Digital Deluxe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,NaN,0,NaN,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,[37944],NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,NaN,[33736],0,169077.0,1621959514,"[2052375, 2057300]",NaN,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### loop through genres list

In [ ]:
genres_dict

In [56]:
# list of genres to add
items = ['point-and-click', 'fighting', 'shooter', 'music', 'platform', 'puzzle', 'racing', 'real-time-strategy-rts', 'role-playing-rpg', 'simulator', 'sport', 'strategy', 'turn-based-strategy-tbs', 'tactical', 'hack-and-slash-beat-em-up', 'quiz-trivia', 'pinball', 'adventure', 'indie', 'arcade', 'visual-novel', 'card-and-board-game', 'moba']
# function to loop through column list and check for genre
def my_list(column, word):
    if word in column:
        return 1
    else:
        return 0

In [57]:
for item in items:
    game_library[item] = game_library['genres'].apply(my_list, word=item)

In [58]:
game_library.head(20)

,id,age_ratings,artworks,category,cover,created_at,external_games,first_release_date,genres,name,...,tactical,hack-and-slash-beat-em-up,quiz-trivia,pinball,adventure,indie,arcade,visual-novel,card-and-board-game,moba
0,192310,"[63703, 63704]",[68886],0,210891.0,1646209573,"[2270833, 2272174, 2326733, 2591087]",1.645661e+09,[adventure],Martha Is Dead: Digital Deluxe,...,0,0,0,0,1,0,0,0,0,0
1,40104,NaN,NaN,0,NaN,1498435200,"[20127, 1988884]",5.364576e+08,[shooter],Dogou Souken,...,0,0,0,0,0,0,0,0,0,0
2,85031,[37944],NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",1.404346e+09,[puzzle],City Mysteries,...,0,0,0,0,0,0,0,0,0,0
3,99234,NaN,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]",1.527120e+09,"[adventure, indie]",Sword of the Black Stone,...,0,0,0,0,1,1,0,0,0,0
4,148930,NaN,[33736],0,169077.0,1621959514,"[2052375, 2057300]",NaN,"[simulator, strategy, adventure, indie]",Bok-Bok: A Chicken Dating Sim,...,0,0,0,0,1,1,0,0,0,0
5,139538,NaN,NaN,0,117024.0,1602368467,[1966278],1.525910e+09,[indie],sun machine,...,0,0,0,0,0,1,0,0,0,0
6,85450,[67799],NaN,0,NaN,1517405808,"[221231, 1748471, 1991656]",NaN,[Not available],Transformers Prime: The Game,...,0,0,0,0,0,0,0,0,0,0
7,33284,NaN,NaN,0,NaN,1495670400,"[5560, 1947044]",1.457741e+09,[indie],One way to exit,...,0,0,0,0,0,1,0,0,0,0
8,89616,NaN,NaN,0,192106.0,1519986874,"[269608, 1949905]",NaN,"[puzzle, arcade]",Bubble Whirl Shooter,...,0,0,0,0,0,0,1,0,0,0
9,104748,NaN,NaN,0,NaN,1530489600,"[1155919, 1972586]",NaN,[Not available],Space station - build your own ISS,...,0,0,0,0,0,0,0,0,0,0


### Adding platforms to game_library

In [62]:
platform_dict

{3: 'Linux',
 4: 'Nintendo 64',
 5: 'Wii',
 6: 'PC (Microsoft Windows)',
 7: 'PlayStation',
 8: 'PlayStation 2',
 9: 'PlayStation 3',
 11: 'Xbox',
 12: 'Xbox 360',
 13: 'DOS',
 14: 'Mac',
 15: 'Commodore C64/128/MAX',
 16: 'Amiga',
 18: 'Nintendo Entertainment System',
 19: 'Super Nintendo Entertainment System',
 20: 'Nintendo DS',
 21: 'Nintendo GameCube',
 22: 'Game Boy Color',
 23: 'Dreamcast',
 24: 'Game Boy Advance',
 25: 'Amstrad CPC',
 26: 'ZX Spectrum',
 27: 'MSX',
 29: 'Sega Mega Drive/Genesis',
 30: 'Sega 32X',
 32: 'Sega Saturn',
 33: 'Game Boy',
 34: 'Android',
 35: 'Sega Game Gear',
 37: 'Nintendo 3DS',
 38: 'PlayStation Portable',
 39: 'iOS',
 41: 'Wii U',
 42: 'N-Gage',
 44: 'Tapwave Zodiac',
 46: 'PlayStation Vita',
 47: 'Virtual Console (Nintendo)',
 48: 'PlayStation 4',
 49: 'Xbox One',
 50: '3DO Interactive Multiplayer',
 51: 'Family Computer Disk System',
 52: 'Arcade',
 53: 'MSX2',
 55: 'Legacy Mobile Device',
 57: 'WonderSwan',
 58: 'Super Famicom',
 59: 'Atari 26

In [63]:
# list of platformd to add
items = ['Linux', 'Nintendo 64', 'Wii', 'PC (Microsoft Windows)', 'PlayStation', 'PlayStation 2', 'PlayStation 3', 'Xbox', 'Xbox 360', 'DOS', 'Mac', 'Nintendo Entertainment System', 'Super Nintendo Entertainment System', 'Nintendo DS', 'Nintendo GameCube', 'Game Boy Color', 'Game Boy Advance', 'Sega Mega Drive/Genesis', 'Sega 32X', 'Sega Saturn', 'Game Boy', 'Android', 'Nintendo 3DS', 'Sega Game Gear', 'PlayStation Portable', 'iOS', 'Wii U', 'PlayStation Vita', 'Virtual Console (Nintendo)', 'PlayStation 4', 'Xbox One', 'Family Computer Disk System', 'Arcade', 'Windows Phone', 'Apple II', 'Web browser', 'Odyssey', 'Commodore 16', 'New Nintendo 3DS']
 

In [64]:
# function to loop through column list and check for genre
def my_list(column, word):
    if word in column:
        return 1
    else:
        return 0

In [ ]:
for item in items:
    game_library[item] = game_library['platforms'].apply(my_list, word=item)

In [ ]:
game_library.head(20)

### Read the CSV for games 

In [ ]:
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
game_library.head()

In [ ]:
game_library = game_library.drop(columns=['websites', 'created_at', 'updated_at', 'artworks', 'cover','screenshots', 'videos', 'checksum_x', 'alternative_names', 'rating_count', 'total_rating_count', 'version_parent', 'hypes', 'franchises', 'follows', 'aggregated_rating_count'])
game_library.shape

In [ ]:
game_library.head()

In [ ]:
game_library.rename(columns={'slug': 'lower_case_name', 'aggregated_rating': 'critic_rating'}, inplace=True)
game_library.head()

### Dataframe with ratings

In [ ]:
df2 = game_library[game_library['rating'].notnull()]
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.rating.isnull().sum()

### Dataframe with nulls in rating column 

In [ ]:
df3 = game_library[game_library['rating'].isnull()]
df3.shape

## Explore

In [ ]:
game_library.head(20)

In [ ]:
game_library.player_perspectives.value_counts()